In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import time
from PIL import Image

In [2]:
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.13.1


In [3]:
path_to_read = 'interpolate_data_low_quality/'
files = []

for r, d, f in os.walk(path_to_read):
    for file in f:
        if '.png' in file:
            files.append(os.path.join(r, file))
files.sort()
np_files = np.asarray(files)

In [4]:
##VERIFICADOR DE TRANSIENTES SEM AS 4 CURVAS
#temp = 'SN000017'
#count = 0
#for f in np_files:
#    if (count == 4):
#        temp = f[29:37]
#        count = 0
#    if ((temp == f[29:37]) & (count < 4)):
#        count += 1
#    elif ((temp != f[29:37]) & (count < 4)):
#        print(f)

In [23]:
#set(types) -> {'IIL', 'IIP', 'II_', 'IIn', 'Ia_', 'Ib_', 'Ibc', 'Ic_'}
conversion = {'IIL' : 5, 'IIP' : 6, 'II_' : 4, 'IIn' : 7, 'Ia_' : 0, 'Ib_' : 1, 'Ibc' : 2, 'Ic_' : 3}

In [6]:
np_grouped = np_files.reshape(round(np_files.shape[0]/4), 4)
class_id = []
for g in np_grouped:
    class_id = class_id + [[g[0][29:37], g[0][38:41]]] 
types = []
for ids in class_id:
    types = types + [conversion[ids[1]]]
set(types)

{1, 2, 3, 4, 5, 6, 7, 8}

In [7]:
astr_objs = []
for i in range(len(np_grouped)):
    astr_objs = astr_objs + [class_id[i]]
    for file in np_grouped[i]:
        img = Image.open(file).convert('L')
        arr = np.array(img)
        astr_objs[-1] = astr_objs[-1] + [arr[5:35,10:56]]
len(astr_objs)


21317

In [8]:
##for ims in astr_objs[0][2:]:
##    print(len(ims[0]))
#
#teste = astr_objs[0][2]
#teste2 = teste[5:35,10:56]
#teste2.shape
#len(teste2[1])

In [24]:
labels = []
images = []
ids = []
for o in astr_objs:
    labels.append((conversion[o[1]])) #converting here to numbers to assure label in training
    images.append(o[2:])
    ids.append(o[0])
labels = np.asarray(labels)
images = np.asarray(images)
ids = np.asarray(ids)

train_n = 18000

train_labels = labels[0:train_n]
test_labels = labels[train_n:]

train_images = images[0:train_n]
test_images = images[train_n:]

train_ids = ids[0:train_n]
test_ids = ids[train_n:]


In [25]:
print(ids.shape, labels.shape, images.shape)

(21317,) (21317,) (21317, 4, 30, 46)


In [26]:
train_images.shape

(18000, 4, 30, 46)

In [27]:
train_images = np.around(train_images/255, decimals = 2)
test_images = np.around(test_images/255, decimals = 2)

# Here starts the training

In [62]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(4, 30, 46)), #aqui mudar o novo shape () 64x40 iu 46x30, x4 talvez 
    #Transform the 2d array into a 1d with mxn 
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(8, activation=tf.nn.softmax) #exit layer, change to the number of class we have ()
])


In [73]:
img_rows, img_cols = 30, 46
input_shape = (4, img_rows, img_cols)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

cnn1 = Sequential()
cnn1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(Dropout(0.2))

cnn1.add(Flatten())

cnn1.add(Dense(128, activation='relu'))
cnn1.add(Dense(10, activation='softmax'))


In [75]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

cnn3 = Sequential()
cnn3.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
cnn3.add(MaxPooling2D((2, 2)))
cnn3.add(Dropout(0.25))

cnn3.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn3.add(MaxPooling2D(pool_size=(2, 2)))
cnn3.add(Dropout(0.25))

cnn3.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn3.add(Dropout(0.4))

cnn3.add(Flatten())

cnn3.add(Dense(128, activation='relu'))
cnn3.add(Dropout(0.3))
cnn3.add(Dense(10, activation='softmax'))

cnn3.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

ValueError: Negative dimension size caused by subtracting 3 from 1 for 'conv2d_10/convolution' (op: 'Conv2D') with input shapes: [?,1,14,32], [3,3,32,64].

In [69]:
cnn1.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [49]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [70]:
cnn1.fit(train_images, train_labels, epochs=10)


Instructions for updating:
Use tf.cast instead.
Epoch 1/10
18000/18000 [==============================] - 7s 389us/step - loss: 1.3326 - acc: 0.5628
Epoch 2/10
18000/18000 [==============================] - 5s 285us/step - loss: 1.3003 - acc: 0.5669
Epoch 3/10
18000/18000 [==============================] - 6s 320us/step - loss: 1.2836 - acc: 0.5685
Epoch 4/10
18000/18000 [==============================] - 6s 355us/step - loss: 1.2826 - acc: 0.5696
Epoch 5/10
18000/18000 [==============================] - 6s 317us/step - loss: 1.2409 - acc: 0.5946
Epoch 6/10
18000/18000 [==============================] - 7s 367us/step - loss: 1.2161 - acc: 0.6023
Epoch 7/10
18000/18000 [==============================] - 8s 460us/step - loss: 1.1880 - acc: 0.6118
Epoch 8/10
18000/18000 [==============================] - 8s 423us/step - loss: 1.1674 - acc: 0.6206
Epoch 9/10
18000/18000 [==============================] - 7s 379us/step - loss: 1.1618 - acc: 0.6228
Epoch 10/10
18000/18000 [==================

In [50]:
model.fit(train_images, train_labels, epochs=10)


Epoch 1/10
18000/18000 [==============================] - 15s 837us/sample - loss: 1.3193 - acc: 0.5633
Epoch 2/10
18000/18000 [==============================] - 17s 958us/sample - loss: 1.2915 - acc: 0.5668
Epoch 3/10
18000/18000 [==============================] - 20s 1ms/sample - loss: 1.2655 - acc: 0.5662
Epoch 4/10
18000/18000 [==============================] - 14s 761us/sample - loss: 1.1352 - acc: 0.6331
Epoch 5/10
18000/18000 [==============================] - 15s 833us/sample - loss: 1.0914 - acc: 0.6563
Epoch 6/10
18000/18000 [==============================] - 17s 964us/sample - loss: 1.0364 - acc: 0.6728
Epoch 7/10
18000/18000 [==============================] - 15s 844us/sample - loss: 1.0212 - acc: 0.6811
Epoch 8/10
18000/18000 [==============================] - 14s 756us/sample - loss: 0.9960 - acc: 0.6907
Epoch 9/10
18000/18000 [==============================] - 14s 784us/sample - loss: 1.0024 - acc: 0.6851
Epoch 10/10
18000/18000 [==============================] - 15s 827

In [51]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

3317/3317 [==============================] - 1s 201us/sample - loss: 0.9887 - acc: 0.6934
Test accuracy: 0.69339764


In [38]:
predictions = model.predict(test_images)

In [46]:
print(predictions[0], '\n',
      'Predicted: ', np.argmax(predictions[0]), ' True one: ', test_labels[0])


[0.23574272 0.07473841 0.01738796 0.07952311 0.4983227  0.02061353
 0.01782578 0.05584575] 
 Predicted:  4  True one:  4


In [35]:
##TEMPLATE DE COMO PEGAR O ARQUIVO
#import numpy as np
#from PIL import Image
#
#img = Image.open('SN000017_II_desg.png').convert('L')
#arr = np.array(img)
#arr = arr[5:35,10:56]
## record the original shape
#shape = arr.shape
#
## make a 1-dimensional view of arr
#flat_arr = arr.ravel()
#
## convert it to a matrix
#vector = np.matrix(flat_arr)
#
## do something to the vector
#vector[:,::10] = 128
#
## reform a numpy array of the original shape
#arr2 = np.asarray(vector).reshape(shape)
#
## make a PIL image
#img2 = Image.fromarray(arr2, 'L')
#img2.show()